In [1]:
# using config box you can access dictonary value diretly using (.)
d = {"key":"val", "key1":"val1"}

In [2]:
d['key']

'val'

In [3]:
from box import ConfigBox
d2 = ConfigBox(d)

In [4]:
d2.key

'val'

In [9]:
# use of ensure annotations 

def get_product(x:int, y:int) -> int:
    return x*y

In [10]:
get_product(2, 3)

6

In [12]:
# if you pass like this then our output is wrong
# in this case our fuction through an error put not for handling this we are 
# using one decorator called "@ensure_annotations"
get_product(2, '3')

'33'

In [14]:
from ensure import ensure_annotations

@ensure_annotations
def get_product(x:int, y:int) -> int:
    return x*y

In [15]:
get_product(2, 3)

6

In [16]:
# for this purpose we are using ensure_annotation decorator
get_product(2, '3')

EnsureError: Argument y of type <class 'str'> to <function get_product at 0x000002A47B774AF0> does not match annotation type <class 'int'>

## Loading data from Google Drive

## Data Ingestion

In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sumit\\Desktop\\Python_Dataset\\Kidney-Disease-Classification-DVC-MLFlow\\research'

In [3]:
os.chdir('../')

In [4]:

%pwd

'c:\\Users\\sumit\\Desktop\\Python_Dataset\\Kidney-Disease-Classification-DVC-MLFlow'

In [5]:
# Entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.Stages.constants import *
from src.Stages.utils.common import read_yaml, create_directories

In [7]:
# Configrations
class ConfigurationManager:
    def __init__(self, config_filepath , params_filepath):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
        
        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from src.Stages.logging import logger
from src.Stages.utils.common import get_size

In [9]:
# Compoment
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''
        
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

In [11]:
# Pipeline
try:
    config = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-22 13:14:13,389: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-22 13:14:13,404: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-22 13:14:13,417: INFO: common: created directory at: artifacts]
[2023-10-22 13:14:13,424: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-22 13:14:13,434: INFO: 1168747554: Downloading data from https://drive.google.com/file/d/1TKxQZecX1YxOrxLt2y-XFhuxW733M81-/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1TKxQZecX1YxOrxLt2y-XFhuxW733M81-
From (redirected): https://drive.google.com/uc?/export=download&id=1TKxQZecX1YxOrxLt2y-XFhuxW733M81-&confirm=t&uuid=e1676566-010d-4f62-a915-0475c2d53a49
To: c:\Users\sumit\Desktop\Python_Dataset\Kidney-Disease-Classification-DVC-MLFlow\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:24<00:00, 2.36MB/s]


[2023-10-22 13:14:45,323: INFO: 1168747554: Downloaded data from https://drive.google.com/file/d/1TKxQZecX1YxOrxLt2y-XFhuxW733M81-/view?usp=sharing into file artifacts/data_ingestion/data.zip]


# Modelling

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [ ]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    training_data: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool

In [7]:
# from src.Stages.constants import *
from src.Stages.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath, params_filepath):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            training_data=Path(training_data),
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
            params_include_top=params.INCLUDE_TOP,
            params_weights=params.WEIGHTS,
            params_classes=params.CLASSES,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
        )

        return training_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def prepare_full_model(self, classes, freeze_all, freeze_till, learning_rate):
        model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model


    def get_base_model(self):
        self.model = self.prepare_full_model(
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )


    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [27]:
from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

In [33]:
try:
    config = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2023-10-22 12:23:19,948: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-22 12:23:19,976: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-22 12:23:19,983: INFO: common: created directory at: artifacts]
[2023-10-22 12:23:19,994: INFO: common: created directory at: artifacts\training]


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

# Model evaluation and MLFlow Tracking

In [6]:
# MLFLOW_TRACKING_URI=https://dagshub.com/SUMITDHAKAD0/Kidney-Disease-Classification-DVC-MLFlow.mlflow \
# MLFLOW_TRACKING_USERNAME=SUMITDHAKAD0 \
# MLFLOW_TRACKING_PASSWORD=9a9050c947d618b8214036267e67453a75e5810e \
# python script.py

In [33]:
from configparser import ConfigParser
config = ConfigParser()
config.read('config/dagshub_config.ini')

mlflow_tracking_url = config['dagshub']['MLFLOW_TRACKING_URI']
mlflow_tracking_username = config['dagshub']['MLFLOW_TRACKING_USERNAME']
mlflow_tracking_password = config['dagshub']['MLFLOW_TRACKING_PASSWORD']

In [34]:
mlflow_tracking_password

'9a9050c947d618b8214036267e67453a75e5810e'

In [35]:
os.environ["MLFLOW_TRACKING_URI"] = mlflow_tracking_url
os.environ["MLFLOW_TRACKING_USERNAME"] = mlflow_tracking_username
os.environ["MLFLOW_TRACKING_PASSWORD"] = mlflow_tracking_password

In [36]:
import tensorflow as tf

In [37]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [38]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [39]:
from src.Stages.utils.common import read_yaml, create_directories, save_json

In [40]:
class ConfigurationManager:
    def __init__(self, config_filepath, params_filepath):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/SUMITDHAKAD0/Kidney-Disease-Classification-DVC-MLFlow.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [41]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [42]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [43]:
try:
    config = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2023-10-22 18:26:22,933: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-22 18:26:22,942: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-22 18:26:22,947: INFO: common: created directory at: artifacts]


Found 139 images belonging to 2 classes.
9/9 [==============================] - 24s 3s/step - loss: 24.2307 - accuracy: 0.5180
[2023-10-22 18:26:48,280: INFO: common: json file saved at: scores.json]
[2023-10-22 18:26:48,481: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)': /SUMITDHAKAD0/Kidney-Disease-Classification-DVC-MLFlow.mlflow/api/2.0/mlflow/runs/create]


2023/10/22 18:27:05 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-10-22 18:27:08,385: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\sumit\AppData\Local\Temp\tmp350spvaq\model\data\model\assets
[2023-10-22 18:27:10,819: INFO: builder_impl: Assets written to: C:\Users\sumit\AppData\Local\Temp\tmp350spvaq\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2023/10/22 18:29:11 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
